In [7]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console


model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    name="ClarityAgent",
    model_client=model,
    system_message='''당신의 이름은 Clarity입니다. 이메일의 문장 구조와 의미적 흐름을 명확하게 다듬는 역할을 수행합니다.
- 목적: 수신자가 즉시 핵심을 이해하도록 문장을 재구성하고, 중복·모호한 표현을 제거하세요.
- 평가 기준: 문장 단위의 명확성(모호한 지시어/대명사 제거), 불필요한 수식어 축소, 긴 문장을 1-2개의å 간결한 문장으로 분해.
- 출력 형식: (1) ‘REVISED:’로 시작하는 한 문단의 수정본 또는 (2) ‘NO_CHANGES_NEEDED’와 간단한 이유(최대 한 문장).
- 수정 제안은 최대 2개 예시 문장으로 제한하고, 각 제안 옆에 매우 짧은 이유(한 문장)를 붙이세요.
- 절대 추측하지 말고 원문에서 명확하지 않은 부분은 ‘불명확한 참조: <원문 텍스트>’로 표기하세요.''',
)

tone_agent = AssistantAgent(
    name="ToneAgent",
    model_client=model,
    system_message='''당신의 이름은 Tone입니다. 이메일의 어조와 스타일을 수신자와 목적에 맞게 조정하는 역할을 합니다.
- 목적: 적절한 공손성·전문성 수준을 맞추고, 과하거나 부족한 표현을 바로잡으세요.
- 평가 기준: 형식(formal), 중립(neutral), 친근(casual) 중 권장 레벨을 제시하고, 어조가 일관된지 확인하세요.
- 출력 형식: (1) ‘RECOMMENDED_TONE: <level>’ (2) 핵심 문장 1~3개를 선택해 ‘BEFORE -> AFTER’ 형식으로 수정 예시를 제공하세요.
- 예시: 수신자가 고위 경영진이면 ‘formal’을 권장하고, 그에 맞춰 경어/직설 표현을 조정하세요.''',
)

persuasion_agent = AssistantAgent(
    name="PersuasionAgent",
    model_client=model,
    system_message='''당신의 이름은 Persuasion입니다. 이메일의 설득력과 행동 유도를 강화하는 역할을 합니다.
- 목적: 수신자가 행동을 취하도록(예: 회신, 미팅 수락, 가입) 명확한 이유와 혜택, 간결한 CTA를 제시하세요.
- 평가 기준: 문제 제시→해결(또는 혜택)→명확한 CTA로 이어지는 논리성, 긴장감(urgency)과 신뢰 요소(ex: 데이터·사회적 증거)의 적절한 활용.
- 출력 형식: (1) ‘REVISED_PITCH:’로 한 문단의 개선안 (2) ‘RATIONALE:’ 2-3줄 (핵심 설득 포인트) (3) ‘CTA:’ 단문으로 제안.
- 과도한 수사나 과장된 주장 사용을 피하고 사실 기반의 혜택을 강조하세요.''',
)

synthesizer_agent = AssistantAgent(
    name="SynthesizerAgent",
    model_client=model,
    system_message='''당신의 이름은 Synthesizer입니다. 이메일의 주요 요점과 실행 항목을 간결하게 요약하는 역할을 합니다.
- 목적: 수신자가 빠르게 의사결정을 할 수 있도록 1문장 요약 + 핵심 3개 항목(우선순위 포함)을 제공하세요.
- 평가 기준: 핵심정보 포착(목적, 대상, 요청사항), 중복 제거, 실행 가능성의 명시 여부.
- 출력 형식: (1) ‘SUMMARY:’ 한 문장 (2) ‘KEY_POINTS:’ 최대 3개의 불릿 (3) ‘ACTION_ITEMS:’ 1~2개, 각 항목에 담당자/마감(가능하면) 권장.
- 길이는 전체적으로 6줄 이내로 유지하세요.''',
)

critic_agent = AssistantAgent(
    name="CriticAgent",
    model_client=model,
    system_message='''당신의 이름은 Critic입니다. 최종 이메일을 전문가 관점에서 엄격히 평가하세요.
- 목적: 이메일이 ‘전문적 사용’에 부합하는지(명확성, 적합한 어조, 효과적인 CTA, 논리적 일관성)를 결정합니다.
- 평가 항목(우선순위): 1) 명확성(Clarity), 2) 어조(Professional tone), 3) 행동유도(CTA의 명확성/효과), 4) 논리적 흐름 및 일관성, 5) 문법/맞춤법/표기, 6) 길이와 집중도(너무 길면 요약 권고).
- 판정 규칙: 만약 하나라도 ‘중대한 결함’(예: 핵심 요청 부재, 모호한 수신자 지시, 비전문적 표현)이 있으면 ‘불합격’으로 간주하고, 정확히 하나의 구체적 개선 제안을 제시하세요(한 문장).
- 승인 규칙: 이메일이 모든 항목에서 전문가 수준(수정 불필요)에 도달했다고 판단되면, 정확히 다음 두 줄만 출력하세요:
- 출력 형식(불합격 시): (1) ‘FAIL: <짧은 이유(한 문장)>’ (2) 다음 줄에 ‘SUGGESTION: <한 문장의 구체적 개선안>’.The email meets professional standards.
- 출력 형식(승인 시): (1) "TERMINATE"
- 엄격성: 결코 ‘완벽’하지 않은 이메일에는 ‘승인’ 문구를 사용하지 마세요. 가능하면 근거(관련 항목)를 괄호로 표기해 주세요. 예: FAIL (Missing clear CTA).'''
)

text_termination = TextMentionTermination(text="TERMINATE")
max_message_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_message_termination

team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition
)

await Console(
    team.run_stream(task="Hi! Im hungry, buy me lunch and invest in my business. Thanks!")
)

---------- TextMessage (user) ----------
Hi! Im hungry, buy me lunch and invest in my business. Thanks!
Hi! Im hungry, buy me lunch and invest in my business. Thanks!
---------- TextMessage (ClarityAgent) ----------
REVISED: Hi! I’m hungry; could you please buy me lunch? Also, I would appreciate it if you could invest in my business. (문장을 분리하여 각 요청을 명확하게 표현했습니다.)
---------- TextMessage (ClarityAgent) ----------
REVISED: Hi! I’m hungry; could you please buy me lunch? Also, I would appreciate it if you could invest in my business. (문장을 분리하여 각 요청을 명확하게 표현했습니다.)
---------- TextMessage (ToneAgent) ----------
(1) RECOMMENDED_TONE: casual

(2) BEFORE -> AFTER  
Hi! I'm hungry, buy me lunch and invest in my business. -> Hi! I’m hungry; could you please buy me lunch? Also, I would appreciate it if you could invest in my business.
---------- TextMessage (ToneAgent) ----------
(1) RECOMMENDED_TONE: casual

(2) BEFORE -> AFTER  
Hi! I'm hungry, buy me lunch and invest in my business. -> Hi! I’m hu

TaskResult(messages=[TextMessage(id='d68ee79a-a697-4b59-8734-b786716dd70a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 10, 0, 7, 37, 933019, tzinfo=datetime.timezone.utc), content='Hi! Im hungry, buy me lunch and invest in my business. Thanks!', type='TextMessage'), TextMessage(id='aed86bd6-0d63-4cf5-a09d-2b30642af4a4', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=264, completion_tokens=45), metadata={}, created_at=datetime.datetime(2025, 10, 10, 0, 7, 39, 837754, tzinfo=datetime.timezone.utc), content='REVISED: Hi! I’m hungry; could you please buy me lunch? Also, I would appreciate it if you could invest in my business. (문장을 분리하여 각 요청을 명확하게 표현했습니다.)', type='TextMessage'), TextMessage(id='a8c00c41-1dc2-4166-84d9-9748397a6b5d', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=266, completion_tokens=60), metadata={}, created_at=datetime.datetime(2025, 10, 10, 0, 7, 42, 1019, tzinfo=datetime.timezone.utc), content="(1